In [1]:
!python3 -m pip install torch torchvision uuid pinecone dotenv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 991.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s et

In [3]:
import os
import uuid
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import pinecone
from pinecone import Pinecone, ServerlessSpec
from PIL import Image

# Vectorization function import
from img2vec import img2vec

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [6]:
img_shape = 1024

In [7]:
# transform = transforms.Compose([
#   transforms.Grayscale(num_output_channels=1),
#   transforms.Resize((img_shape, img_shape)),
#   transforms.ToTensor(),
# ])

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.225, 0.225, 0.225])
])

In [8]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100.0%


In [9]:
vec_dimensions = img2vec(train_dataset[0][0]).shape[0]

In [10]:
vec_dimensions

2048

In [11]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = f"cifar10-index-{vec_dimensions}"

# Index has already been created!
# pc.create_index(
#   name=index_name,
#   dimension=vec_dimensions,
#   metric="cosine",
#   spec=ServerlessSpec(
#     cloud="aws",
#     region="us-east-1"
#   )
# )

cifar_index = pc.Index(index_name)

In [12]:
cifar_index.describe_index_stats()

{'dimension': 2048,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 2000}},
 'total_vector_count': 2000,
 'vector_type': 'dense'}

In [14]:
#cifar_index.delete(delete_all=True)

In [19]:
# Training process: Query train dataset vectors on vector database and if
# response is empty or the ANNs are less than 80%, upsert query vector to
# vector database.

In [ ]:
for i in range(len(train_dataset)):
  current_img_vec = img2vec(train_dataset[i][0])
  cifar_index.upsert(vectors=[(str(uuid.uuid4()), current_img_vec, {"class": int(train_dataset[i][1])})])
  if (i + 1) % 50 == 0:
    print(f"Uploaded vector {i+1}/{len(train_dataset)}")

Uploaded vector 50/50000
Uploaded vector 100/50000
Uploaded vector 150/50000
Uploaded vector 200/50000
Uploaded vector 250/50000
Uploaded vector 300/50000
Uploaded vector 350/50000
Uploaded vector 400/50000
Uploaded vector 450/50000
Uploaded vector 500/50000
Uploaded vector 550/50000
Uploaded vector 600/50000
Uploaded vector 650/50000
Uploaded vector 700/50000
Uploaded vector 750/50000
Uploaded vector 800/50000
Uploaded vector 850/50000
Uploaded vector 900/50000
Uploaded vector 950/50000
Uploaded vector 1000/50000
Uploaded vector 1050/50000
Uploaded vector 1100/50000
Uploaded vector 1150/50000
Uploaded vector 1200/50000
Uploaded vector 1250/50000
Uploaded vector 1300/50000
Uploaded vector 1350/50000
Uploaded vector 1400/50000
Uploaded vector 1450/50000
Uploaded vector 1500/50000
Uploaded vector 1550/50000
Uploaded vector 1600/50000
Uploaded vector 1650/50000
Uploaded vector 1700/50000
Uploaded vector 1750/50000
Uploaded vector 1800/50000
Uploaded vector 1850/50000
Uploaded vector 1900/

In [ ]:
# # Training process

# for i in range(len(train_dataset)):
#   current_query_vector = img2vec(train_dataset[i][0]).tolist()

#   response = cifar_index.query(
#     namespace="",
#     vector=current_query_vector,
#     top_k=5,
#     include_values=False,
#     include_metadata=True,
#   )

#   avg_score = 0

#   for j in range(len(response["matches"])):
#     avg_score += response["matches"][j]["score"]

#   if len(response["matches"]) != 0:
#     avg_score /= len(response["matches"])

#   if avg_score < 0.80:
#     cifar_index.upsert(vectors=[(str(uuid.uuid4()), current_query_vector, {"class": int(train_dataset[i][1])})])
#     print(f"Trained on image {i+1}/{len(train_dataset)}")

In [ ]:
accuracy = 0
test_dataset_size = len(test_dataset)

for i in range(len(test_dataset)):
  test_query_vector = img2vec(test_dataset[i][0])
  label = test_dataset[i][1]

  response = cifar_index.query(
    namespace="",
    vector=test_query_vector,
    top_k=1,
    include_values=False,
    include_metadata=True,
  )

  pred = int(response["matches"][0]["metadata"]["class"])
  accuracy += 1 if (pred == label) else 0
  print(f"Current Accuracy: {accuracy / (i+1)}")

print()
print()
print()
print()
print(f"Accuracy: {accuracy / test_dataset_size:.2f}")


Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.0
Current Accuracy: 0.043478260869565216
Current Accuracy: 0.041666666666666664
Current Accuracy: 0.04
Current Accuracy: 0.038461538461538464
Current Accuracy: 0.07407407407407407
Current Accuracy: 0.07142857142857142
Current Accuracy: 0.06896551724137931
Current Accuracy: 0.06666666666666667
Current Accuracy: 0.06451612903225806
Current Accuracy: 0.0625
Current Accuracy: 0.09090909090909091
Current Accuracy: 0.08823529411764706
Current Accuracy: 0.08571428571428572
Current Accuracy: 0.08333333333333333
Current A

KeyboardInterrupt: 

In [ ]:
print(f"Accuracy: {accuracy / test_dataset_size * 100}%")

In [ ]:
accuracy

1000

In [ ]:
test_dataset_size

10000